<a href="https://colab.research.google.com/github/RafsanJany-44/ARC/blob/master/VitalDB_ECG_MBP_Feature_extraction_09_30_2023_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:

!pip install pyvital vitaldb vital
!pip install neurokit2
!pip install -q wfdb
!pip install -q neurokit2
!pip install -q biosppy

import vitaldb
import pandas as pd
import numpy as np
from datetime import timedelta
import neurokit2 as nk
import matplotlib.pyplot as plt
import os
import pandas as pd
import biosppy.signals.ecg as ecg
import random
from tqdm.notebook import tqdm


df_cases = pd.read_csv("https://api.vitaldb.net/cases")  # Load clinical data
df_trks = pd.read_csv('https://api.vitaldb.net/trks')  # Load track list
df_labs = pd.read_csv('https://api.vitaldb.net/labs')  # Load lab result

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.5/202.5 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.7/824.7 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.8/428.8 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 6.8 

In [12]:
u'Δ_x,y'

'Δ_x,y'

In [13]:
def r_wave_amplitude(r_peak_loc):
    if len(r_peak_loc) >= 1:
        r_amplitude = float(r_peak_loc.values[0])
        return r_amplitude
    return np.nan

def r_wave_duration(q_peak_loc, t_onset_loc):
    if len(q_peak_loc) >= 1 and len(t_onset_loc) >= 1:
        r_duration = float(q_peak_loc.index[0] - t_onset_loc.index[0])
        return r_duration
    return np.nan

def t_wave_amplitude(t_peak_loc):
    if len(t_peak_loc) >= 1:
        t_amplitude = float(t_peak_loc.values[0])
        return t_amplitude
    return np.nan

def t_wave_duration(t_offset_loc, t_onset_loc):
    if len(t_offset_loc) >= 1 and len(t_onset_loc) >= 1:
        t_duration = float(t_offset_loc.index[0] - t_onset_loc.index[0])
        return t_duration
    return np.nan


def p_wave_duration(p_offset_loc, p_onset_loc):
    if len(p_offset_loc) >= 1 and len(p_onset_loc) >= 1:
        p_duration = float(p_offset_loc.index[0] - p_onset_loc.index[0])
        return p_duration
    return np.nan


def q_wave_amplitude(q_peak_loc):
    if len(q_peak_loc) >= 1:
        q_amplitude = float(q_peak_loc.values[0])
        return q_amplitude
    return np.nan

def s_wave_amplitude(s_peak_loc):
    if len(s_peak_loc) >= 1:
        s_amplitude = float(s_peak_loc.values[0])
        return s_amplitude
    return np.nan

def qt_interval(t_offset_loc, p_offset_loc):
    if len(t_offset_loc) >= 1 and len(p_offset_loc) >= 1:
        qt_interval = float(t_offset_loc.index[0] - p_offset_loc.index[0])
        return qt_interval
    return np.nan


def qr_interval(r_offset_loc, p_offset_loc):
    if len(r_offset_loc) >= 1 and len(p_offset_loc) >= 1:
        qt_interval = float(r_offset_loc.index[0] - p_offset_loc.index[0])
        return qr_interval
    return np.nan

def heart_rate_calc(hr, idx):
    if idx > 0:
        return float(hr[idx-1])
    return np.nan


def j_point_calc(s_peak_loc, t_peak_loc, x, y):
    if len(s_peak_loc) == 0 or len(t_peak_loc) == 0:
        return np.nan, np.nan, None
    if s_peak_loc.index[0] > t_peak_loc.index[0]:
        return np.nan, np.nan, None
    s_peak_ind = int(np.interp(s_peak_loc.index[0], x, np.arange(len(x))))
    t_peak_ind = int(np.interp(t_peak_loc.index[0], x, np.arange(len(x))))
    y_st_by_peaks = y[s_peak_ind:t_peak_ind+1]
    x_st_by_peaks = x[s_peak_ind:t_peak_ind+1]
    if t_peak_ind - s_peak_ind < 1:
        return np.nan, np.nan, None
    Fy = np.gradient(y_st_by_peaks)
    j_point_pseudo_ind = np.abs(Fy).argmin()
    j_point = [x_st_by_peaks[j_point_pseudo_ind], y_st_by_peaks[j_point_pseudo_ind]]
    j_point_ind = int(np.round(np.interp(x_st_by_peaks[j_point_pseudo_ind], x, np.arange(len(x)))))
    j_magnitude = j_point[1]

    return j_point, j_magnitude, j_point_ind

def st_duration_calc(t_peak_loc, j_point):
    if len(t_peak_loc) >= 1 and isinstance(j_point, list) and len(j_point) == 2:
        st_duration = t_peak_loc.index[0] - j_point[0]
        return st_duration
    return np.nan

def st_slope_calc(t_onset_loc, j_point_ind, x, y):
    if len(t_onset_loc) >= 1 and j_point_ind is not None:
        t_onset_ind = int(np.round(np.interp(t_onset_loc.index[0], x, np.arange(len(x)))))

        if t_onset_ind > j_point_ind:
            y_st_slope_range = y[j_point_ind:t_onset_ind+1]
            x_st_slope_range = x[j_point_ind:t_onset_ind+1]

            F_st_slope = np.gradient(y_st_slope_range)
            st_slope = np.mean(F_st_slope)
            st_slope_bool = st_slope > 0
            return st_slope
    return np.nan

def st_area_calc(t_offset_loc, j_point_ind, x, y,):
    if len(t_offset_loc) >= 1 and j_point_ind is not None:
        ind_t_offset = int(np.round(np.interp(t_offset_loc.index[0], x, np.arange(len(x)))))
        y_st_area_range = y[j_point_ind:ind_t_offset+1]
        x_st_area_range = x[j_point_ind:ind_t_offset+1]
        st_area = np.trapz(y_st_area_range, x_st_area_range)
        return st_area
    return np.nan





def extract_features(beats, features,case_id):

    p_onsets = features["ECG_P_Onsets"]
    r_onsets = features["ECG_R_Onsets"]
    t_onsets = features["ECG_T_Onsets"]
    p_offsets = features["ECG_P_Offsets"]
    r_offsets = features["ECG_R_Offsets"]
    t_offsets = features["ECG_T_Offsets"]
    r_peaks = features["ECG_R_Peaks"]
    t_peaks = features["ECG_T_Peaks"]
    q_peaks = features["ECG_Q_Peaks"]
    s_peaks = features["ECG_S_Peaks"]
    p_peaks = features["ECG_P_Peaks"]
    heart_rate = features["ECG_Rate"]
    cycles=0

    record_dataset = pd.DataFrame(columns=['Case_ID','Cycle','R_wave_amplitude',u'ΔR_wave_amplitude',
                                           'R_wave_duration',u'ΔR_wave_duration',
                                           'T_wave_amplitude',u'ΔT_wave_amplitude',
                                           'T_wave_duration',u'ΔT_wave_duration',
                                           'p_wave_duration',u'ΔP_wave_duration',
                                           'S_wave_amplitude',u'ΔS_wave_amplitude',
                                           'ST_amplitude',u'ΔST_amplitude',
                                           'q_wave_amplitude',u'Δq_wave_amplitude',
                                           'QT_interval',u'ΔQT_interval',
                                           'Heart_rate',u'ΔHeart_rate',
                                           'J_point_amplitude',u'ΔJ_point_amplitude',
                                           'ST_duration',u'ΔST_duration',
                                           'ST_area', u'ΔST_area',
                                           'ST_slope',u'ΔST_slope'])

    for i in range(1, len(beats)+1):
        beat = beats[str(i)]
        beat_signal = beat['Signal']
        time = beat_signal.index
        voltage = beat_signal.values
        beat_idx = beat['Index'].values
        min_idx, max_idx = beat_idx[0], beat_idx[-1]

        s_peak_loc = beat_signal.iloc[np.argwhere(s_peaks[min_idx:max_idx].values == 1).flatten()]
        r_onset_loc = beat_signal.iloc[np.argwhere(r_onsets[min_idx:max_idx].values == 1).flatten()]
        t_onset_loc = beat_signal.iloc[np.argwhere(t_onsets[min_idx:max_idx].values == 1).flatten()]
        p_offset_loc = beat_signal.iloc[np.argwhere(p_offsets[min_idx:max_idx].values == 1).flatten()]
        p_onset_loc = beat_signal.iloc[np.argwhere(p_onsets[min_idx:max_idx].values == 1).flatten()]
        r_offset_loc = beat_signal.iloc[np.argwhere(r_offsets[min_idx:max_idx].values == 1).flatten()]
        t_offset_loc = beat_signal.iloc[np.argwhere(t_offsets[min_idx:max_idx].values == 1).flatten()]
        r_peak_loc = beat_signal.iloc[np.argwhere(r_peaks[min_idx:max_idx].values == 1).flatten()]
        t_peak_loc = beat_signal.iloc[np.argwhere(t_peaks[min_idx:max_idx].values == 1).flatten()]
        q_peak_loc = beat_signal.iloc[np.argwhere(q_peaks[min_idx:max_idx].values == 1).flatten()]


        p_duration = p_wave_duration(p_offset_loc, p_onset_loc)
        r_amplitude = r_wave_amplitude(r_peak_loc)
        r_duration = r_wave_duration(q_peak_loc, r_onset_loc)
        t_amplitude = t_wave_amplitude(t_peak_loc)
        t_duration = t_wave_duration(t_offset_loc, t_onset_loc)
        s_amplitude = s_wave_amplitude(s_peak_loc)
        st_amplitude = s_amplitude + t_amplitude
        q_amplitude = q_wave_amplitude(q_peak_loc)
        qt_int = qt_interval(t_offset_loc, p_offset_loc)
        #qr_int = qr_interval(r_offset_loc, p_offset_loc)
        hr = heart_rate_calc(heart_rate, (min_idx + max_idx) // 2)
        j_point, j_magnitude, j_point_ind = j_point_calc(s_peak_loc, t_peak_loc, time, voltage)
        st_duration = st_duration_calc(t_peak_loc, j_point)
        st_slope = st_slope_calc(t_onset_loc, j_point_ind, time, voltage)
        st_area = st_area_calc(t_offset_loc, j_point_ind, time, voltage)
        cycles+=1

        feature_row = {
            'Case_ID': case_id,
            'Cycle': cycles,
            'R_wave_amplitude': r_amplitude,
             u'ΔR_wave_amplitude': r_amplitude,
            'R_wave_duration': r_duration,
             u'ΔR_wave_duration': r_duration,
            'T_wave_amplitude': t_amplitude,
             u'ΔT_wave_amplitude': t_amplitude,
            'T_wave_duration': t_duration,
             u'ΔT_wave_duration': t_duration,
            'p_wave_duration': p_duration,
             u'ΔP_wave_duration': p_duration,
            'S_wave_amplitude': s_amplitude,
             u'ΔS_wave_amplitude': s_amplitude,
            'ST_amplitude': st_amplitude,
             u'ΔST_amplitude': st_amplitude,
            'q_wave_amplitude': q_amplitude,
             u'Δq_wave_amplitude': q_amplitude,
            'QT_interval': qt_int,
             u'ΔQT_interval': qt_int,
            'Heart_rate': hr,
             u'ΔHeart_rate': hr,
            'J_point_amplitude': j_magnitude,
             u'ΔJ_point_amplitude': j_magnitude,
            'ST_duration': st_duration,
             u'ΔST_duration': st_duration,
            'ST_area': st_area,
             u'ΔST_area': st_area,
            'ST_slope': st_slope,
             u'ΔST_slope': st_slope
        }

        feature_row = pd.DataFrame([feature_row])
        record_dataset = pd.concat([record_dataset, feature_row], ignore_index=True)

    return record_dataset






def engine(initial_signal,caseid):
  srate = 100
  initial_signal = initial_signal[np.logical_not(np.isnan(initial_signal))]
  initial_signal = initial_signal[(initial_signal>= -.25) & (initial_signal<=.25)]
  signal_clean = nk.ecg_clean(initial_signal, sampling_rate=srate)
  features, _ = nk.ecg_process(signal_clean, sampling_rate=srate)
  beats = nk.ecg_segment(signal_clean, sampling_rate=srate)
  data = extract_features(beats, features,caseid)
  return data



In [6]:
caseids = list(
    set(df_trks[df_trks['tname'] == 'Solar8000/ST_II']['caseid']) &
    set(df_trks[df_trks['tname'] == 'Solar8000/ART_MBP']['caseid']) &
     set(df_trks[df_trks['tname'] == 'Solar8000/ART_SBP']['caseid'])&
     set(df_trks[df_trks['tname'] == 'Solar8000/ART_DBP']['caseid'])&
    set(df_trks[df_trks['tname'] == 'SNUADC/ECG_II']['caseid'])
)

print('Total {} cases found'.format(len(caseids)))

Total 3559 cases found


In [7]:
caseids = caseids[0:4]
caseids

[1, 3, 4, 7]

In [14]:
file_save_destination = '/content/Vital_ECG_II_features.csv'

#file_save_destination = '/content/drive/MyDrive/Cornell/MADE Lab/VitalDB/Data/VitalDB ECG Features/Vital_ECG_MAP_features_600s.csv'


if os.path.exists(file_save_destination)==False:

  main_dataframe = pd.DataFrame(columns=['Case_ID', 'Segment', 'ST_II', 'ART_MBP', 'ART_SBP', 'ART_DBP', 'Cycle',
       'R_wave_amplitude',u'ΔR_wave_amplitude', 'R_wave_duration',u'ΔR_wave_duration',
       'T_wave_amplitude',u'ΔT_wave_amplitude', 'T_wave_duration',u'ΔT_wave_duration',
       'p_wave_duration',u'ΔP_wave_duration', 'S_wave_amplitude',u'ΔS_wave_amplitude',
       'ST_amplitude',u'ΔST_amplitude', 'q_wave_amplitude',u'Δq_wave_amplitude',
       'QT_interval',u'ΔQT_interval', 'Heart_rate',u'ΔHeart_rate',
       'J_point_amplitude',u'ΔJ_point_amplitude', 'ST_duration',u'ΔST_duration',
       'ST_area', u'ΔST_area','ST_slope',u'ΔST_slope'])

  main_dataframe.to_csv(file_save_destination,index=False)

main_dataframe = pd.read_csv(file_save_destination)
case_complete_list = list(main_dataframe["Case_ID"].unique())




for caseid in tqdm(caseids):

  if caseid not in case_complete_list:
    try:
      print("Case Id:", caseid,"in process")
      vals = vitaldb.load_case(caseid, ['SNUADC/ECG_II','Solar8000/ST_II','Solar8000/ART_MBP','Solar8000/ART_SBP','Solar8000/ART_DBP'], 1 / 100)

      sub_dataframe = pd.DataFrame(columns=['Case_ID', 'Segment', 'ST_II', 'ART_MBP', 'ART_SBP', 'ART_DBP', 'Cycle',
            'R_wave_amplitude',u'ΔR_wave_amplitude', 'R_wave_duration',u'ΔR_wave_duration',
            'T_wave_amplitude',u'ΔT_wave_amplitude', 'T_wave_duration',u'ΔT_wave_duration',
            'p_wave_duration',u'ΔP_wave_duration', 'S_wave_amplitude',u'ΔS_wave_amplitude',
            'ST_amplitude',u'ΔST_amplitude', 'q_wave_amplitude',u'Δq_wave_amplitude',
            'QT_interval',u'ΔQT_interval', 'Heart_rate',u'ΔHeart_rate',
            'J_point_amplitude',u'ΔJ_point_amplitude', 'ST_duration',u'ΔST_duration',
            'ST_area', u'ΔST_area','ST_slope',u'ΔST_slope'])

      op_st = df_cases[(df_cases['caseid'] == caseid)]['opstart'].values[0]
      op_end = df_cases[(df_cases['caseid'] == caseid)]['opend'].values[0]

      time_col=[]
      for i in range(len(vals)):
        time_col.append(i*(1/500))



      vals = np.insert(vals, 0, time_col, axis=1)
      vals = np.insert(vals, 0, np.nan, axis=1)


      vals  = vals[vals[:,1] >= op_st]
      vals =  vals[vals[:,1] <= op_end]


      sec_segment = 120
      segment_num = 1
      start_idx = 0
      end_idx = int(sec_segment/(1/500) + 1)

      for _ in tqdm(range(int(vals.shape[0]/int(sec_segment/(1/500) + 1)))):
          try:
            print("Case_ID:",caseid ," Segment:",segment_num)
            temp_arr = vals[start_idx:end_idx,:]

            df = engine(temp_arr[:,2],caseid)
            arr1 = np.nanmean(temp_arr, axis = 0)

            map = (arr1[5]+2*arr1[6])/3
            if map>= 75:
              stat = "Non-Hypotension"
            elif map<=65:
              stat = "Hypotension"
            elif 65<map<75:
              stat = "Intermediate"
            else:
              stat = np.nan

            df.insert(1, 'Status', stat)
            df.insert(1, 'ART_MAP', map)
            df.insert(1, 'ART_DBP', arr1[6])
            df.insert(1, 'ART_SBP', arr1[5])
            df.insert(1, 'ART_MBP',  arr1[4])
            df.insert(1, 'ST_II', arr1[3])
            df.insert(1, 'Segment', segment_num)

            sub_dataframe = pd.concat([sub_dataframe, df], ignore_index=True)

            start_idx = end_idx
            end_idx = end_idx + int(sec_segment/(1/500))
            segment_num += 1

          except:
            start_idx = end_idx
            end_idx = end_idx + int(sec_segment/(1/500))
            print("--------ERORR------!!!")

      main_dataframe = pd.concat([main_dataframe, sub_dataframe], ignore_index=True)
      main_dataframe.to_csv(file_save_destination,index=False)
      print("Case Id:", caseid," processing complete.....")
    except:
      print("----------------------------ERORR!!! occurred In CASE ID: ",caseid,"----------------------------------")
  else:
    print("CASE_ID ", caseid," already exists....")


  0%|          | 0/4 [00:00<?, ?it/s]

Case Id: 1 in process


  0%|          | 0/5 [00:00<?, ?it/s]

Case_ID: 1  Segment: 1


<ipython-input-14-e057db8d4362>:69: RuntimeWarning: Mean of empty slice
  arr1 = np.nanmean(temp_arr, axis = 0)


...

Case_ID: 4  Segment: 12
--------ERORR------!!!
Case_ID: 4  Segment: 12


<ipython-input-14-e057db8d4362>:69: RuntimeWarning: Mean of empty slice
  arr1 = np.nanmean(temp_arr, axis = 0)


Case Id: 4  processing complete.....
Case Id: 7 in process


0it [00:00, ?it/s]

Case Id: 7  processing complete.....


In [15]:
main_dataframe

,Case_ID,Segment,ST_II,ART_MBP,ART_SBP,ART_DBP,Cycle,R_wave_amplitude,ΔR_wave_amplitude,R_wave_duration,...,J_point_amplitude,ΔJ_point_amplitude,ST_duration,ΔST_duration,ST_area,ΔST_area,ST_slope,ΔST_slope,ART_MAP,Status
0,1,1,-0.407122,123.596667,157.458647,79.593985,1,NaN,NaN,0.101056,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.548872,Non-Hypotension
1,1,1,-0.407122,123.596667,157.458647,79.593985,2,0.094657,0.094657,NaN,...,-0.057246,-0.057246,0.020211,0.020211,0.000751,0.000751,NaN,NaN,105.548872,Non-Hypotension
2,1,1,-0.407122,123.596667,157.458647,79.593985,3,0.084247,0.084247,0.020211,...,-0.072725,-0.072725,0.171796,0.171796,0.000706,0.000706,0.005674,0.005674,105.548872,Non-Hypotension
3,1,1,-0.407122,123.596667,157.458647,79.593985,4,-0.035654,-0.035654,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.548872,Non-Hypotension
4,1,1,-0.407122,123.596667,157.458647,79.593985,5,0.098648,0.098648,NaN,...,-0.045366,-0.045366,0.020211,0.020211,-0.001729,-0.001729,NaN,NaN,105.548872,Non-Hypotension
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12160,4,12,-0.946000,89.040000,141.555556,57.496296,447,0.102835,0.102835,0.090482,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.516049,Non-Hypotension
12161,4,12,-0.946000,89.040000,141.555556,57.496296,448,0.115188,0.115188,0.080428,...,-0.130670,-0.130670,0.170910,0.170910,0.006350,0.006350,0.011574,0.011574,85.516049,Non-Hypotension
12162,4,12,-0.946000,89.040000,141.555556,57.496296,449,0.007264,0.007264,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.516049,Non-Hypotension
12163,4,12,-0.946000,89.040000,141.555556,57.496296,450,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.516049,Non-Hypotension


In [ ]:
#main_dataframe.to_csv(file_save_destination,index=False)

In [ ]:
'''
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Cornell/MADE Lab/VitalDB/Data/VitalDB ECG Features/Vital_ECG_MAP_features_09.30.2023.csv')
df.shape
'''

In [ ]:
#main_dataframe